In [1]:
import os
import torch
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

cuda


In [2]:
import os 
import json
import re

import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np
from datasets import Dataset, DatasetDict, concatenate_datasets
from datasets import Features, Value
from datasets import  load_dataset, DownloadConfig
from collections import deque,defaultdict
import pickle
from datasets import load_from_disk
import pandas as pd
from src.utils.data_utils import format_data_sfsa
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import collections
from datetime import datetime
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
import torch.nn.functional as F

from pettingzoo.mpe import simple_adversary_v3
from src.utils import rl_tools
from models.maddpg.e2t_maddpg import ENMADDPG
from models.utils import persistence
from src.utils.data_utils import find_latest_file
from src.utils.model_utils import pad_to_shape


In [3]:
max_cycles = 200
seed = 42

num_episodes = 100
episode_length = 64 
buffer_size = 100000
hidden_dim = 64
actor_lr = 1e-2
critic_lr = 1e-2
gamma = 0.95
tau = 1e-2
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
update_interval = 32
minimal_size = 15
replay_buffer = rl_tools.ReplayBuffer(buffer_size)


cuda


# 1. Building EN_maddpg

In [4]:
en_out_dim = 768
redu_dim = 16
max_length = 128

In [5]:
max_agents = 6

In [6]:
def add_en_dim(state_dim, redu_dim):

    en_state_dims = []
    for x in state_dim:
        en_state_dims.append(x + redu_dim)
    return en_state_dims

In [7]:
state_dims=[34, 34, 34, 34, 34, 34] 
action_dims=[50, 50, 50, 50, 50, 50] 

en_state_dims = add_en_dim(state_dims, redu_dim)


print(en_state_dims)
print(action_dims)

[50, 50, 50, 50, 50, 50]
[50, 50, 50, 50, 50, 50]


In [8]:
critic_input_dim = sum(state_dims) +sum(action_dims) 

In [9]:
print(critic_input_dim)

504


In [10]:
encoders_dir = './parameters/weights/encoders/bert_EN'
spec_en = ''

In [11]:
encoder = torch.load(spec_en) if spec_en else torch.load(find_latest_file(encoders_dir))

In [12]:
tokenizer = "bert-base-uncased"
tokenizer_fn = "tokenizer_fn"

In [13]:
en_maddpg = ENMADDPG(
    tokenizer, 
    tokenizer_fn,
    max_length,
    encoder, 
    en_out_dim, 
    redu_dim, 
    max_agents, 
    device, 
    actor_lr, 
    critic_lr, 
    hidden_dim, 
    state_dims, 
    action_dims, 
    critic_input_dim, 
    gamma, 
    tau
    )
en_maddpg = en_maddpg.to(device)

ENDDPG 768 16 34 50 504 64 0.01 0.01 cuda
ENDDPG 768 16 34 50 504 64 0.01 0.01 cuda
ENDDPG 768 16 34 50 504 64 0.01 0.01 cuda
ENDDPG 768 16 34 50 504 64 0.01 0.01 cuda
ENDDPG 768 16 34 50 504 64 0.01 0.01 cuda
ENDDPG 768 16 34 50 504 64 0.01 0.01 cuda


In [14]:

print(sum(p.numel() for p in en_maddpg.parameters())/1e9,'B parameters in en_maddpg')



0.660155844 B parameters in en_maddpg


# 2. Training EN_MADDPG

In [15]:
#rl_mpe_env_raw = load_dataset("graycatHCO3/RL_MPE_env_raw")
rl_mpe_env_raw = load_from_disk('./data/HuggingFace/RL_MPE_env_raw');

In [16]:
#rl_mpe_env_raw.save_to_disk('./data/HuggingFace/RL_MPE_env_raw')

In [17]:
rl_mpe_env_raw['RL_MPE_raw']['Env_name']

['simple.txt',
 'simple_adversary.txt',
 'simple_crypto.txt',
 'simple_push.txt',
 'simple_reference.txt',
 'simple_speaker_listener.txt',
 'simple_spread.txt',
 'simple_tag.txt',
 'simple_world_comm.txt']

In [18]:
desc = rl_mpe_env_raw['RL_MPE_raw']['content'][1]

In [19]:
len(en_maddpg.agents)


6

In [20]:
used_agents = 3

In [21]:
env = simple_adversary_v3.parallel_env(max_cycles=max_cycles)
observations, infos = env.reset() if seed == -1 else env.reset(seed=seed)

env_action_dims = []
for action_space_key, action_space in env.action_spaces.items():
    env_action_dims.append(action_space.n)

env_action_dims

d:\Disk_D\programming_software\AI\Anaconda3\envs\pettingZoo_Langchain\lib\site-packages\pettingzoo\utils\conversions.py:158: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn(


[5, 5, 5]

In [22]:
return_list = [] 
total_step = 0
replay_buffer = rl_tools.ReplayBuffer(buffer_size)

agents = ['adversary_0', 'agent_0', 'agent_1']



for i_episode in tqdm(range(num_episodes)):   
   
    state, info = env.reset() if seed == -1 else env.reset(seed=seed)
    #print(state)
    state_padded = {
        agent_name: pad_to_shape(state[agent_name], agent.obs_shape)
        for agent_name, agent in zip(state.keys(), en_maddpg.agents)
    }
    for e_i in range(episode_length):   
        #print(type(desc))
        actions = en_maddpg.take_action(state_padded, desc,env_action_dims, used_agents, explore=True)
     
        env_actions = {agent: np.argmax(action) for agent, action in zip(agents, actions)}

        next_state, reward, done, truncations, infos = env.step(env_actions)
        #print(next_state)
        next_state_padded = {
            agent_name: pad_to_shape(next_state[agent_name], agent.obs_shape)
            for agent_name, agent in zip(next_state.keys(), en_maddpg.agents)
        }
        actions_padded = {
            agent_name: pad_to_shape(actions[i], agent.action_shape)
            for (i,agent_name), agent in zip(enumerate(agents), en_maddpg.agents)
        }
        #print(actions_padded)
        replay_buffer.add(state_padded, actions_padded, reward, next_state_padded, done)
        state_padded = next_state_padded

        total_step += 1

        if replay_buffer.size() >= minimal_size and total_step % update_interval == 0:
            obs, act, rew, next_obs, done = replay_buffer.sample(batch_size)
            #print(obs)
            obs_list, act_list, rew_list, next_obs_list, done_list = [], [], [], [], []

            for i, agent in enumerate(['adversary_0', 'agent_0', 'agent_1']):
                obs_list.append(np.array([s[agent] for s in obs]))
                #print(act)
                act_list.append(np.array([a[agents[i]] for a in act]))
                rew_list.append(np.array([r[agent] for r in rew]))
                next_obs_list.append(np.array([ns[agent] for ns in next_obs]))
                done_list.append(np.array([d[agent] for d in done]))

            obs_tensor = [torch.tensor(o, dtype=torch.float32).to(device) for o in obs_list]
            act_tensor = [torch.tensor(a, dtype=torch.float32).to(device) for a in act_list]
            rew_tensor = [torch.tensor(r, dtype=torch.float32).to(device) for r in rew_list]                 
            next_obs_tensor = [torch.tensor(n, dtype=torch.float32).to(device) for n in next_obs_list]
            done_tensor = [torch.tensor(d, dtype=torch.float32).to(device) for d in done_list]  # 将布尔类型转换为浮点数类型


            for a_i in range(len(env.agents)):
                en_maddpg.update(obs_tensor,act_tensor,rew_tensor,next_obs_tensor,done_tensor, a_i, desc, env_action_dims)#(要素，agent编号) #感觉可以改成一次传参优化速度
            en_maddpg.update_all_targets()


    ep_returns = rl_tools.en_evaluate(env, en_maddpg,desc, env_action_dims,used_agents, n_episode=100)
    return_list.append(ep_returns)
   
    print("actions:", env_actions)
    print(f"Episode: {i_episode+1}, {ep_returns}")
  
             
env.close()

current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

return_array = np.array(return_list)

return_info = np.array([max_cycles,seed,num_episodes])

np.savez(f'././evaluation/results/e2t_maddpg/{current_time}.npz',return_array=return_array, return_info=return_info) 





  1%|          | 1/100 [23:51<39:21:20, 1431.12s/it]

actions: {'adversary_0': 0, 'agent_0': 3, 'agent_1': 0}
Episode: 1, [1.3914398113762354, 1.3914398113762354, 1.3914398113762354, 1.3914398113762354, 1.3914398113762354, 1.3914398113762354]


  2%|▏         | 2/100 [47:40<38:56:05, 1430.26s/it]

actions: {'adversary_0': 3, 'agent_0': 0, 'agent_1': 1}
Episode: 2, [-4.119090953003203, -4.119090953003203, -4.119090953003203, -4.119090953003203, -4.119090953003203, -4.119090953003203]


  3%|▎         | 3/100 [1:11:31<38:32:27, 1430.39s/it]

...

actions: {'adversary_0': 4, 'agent_0': 0, 'agent_1': 4}
Episode: 10, [-3.6663590955529592, -3.6663590955529592, -3.6663590955529592, -3.6663590955529592, -3.6663590955529592, -3.6663590955529592]


 11%|█         | 11/100 [4:22:18<35:22:59, 1431.23s/it]

actions: {'adversary_0': 4, 'agent_0': 0, 'agent_1': 2}
Episode: 11, [-8.109762073505614, -8.109762073505614, -8.109762073505614, -8.109762073505614, -8.109762073505614, -8.109762073505614]


 11%|█         | 11/100 [4:36:32<37:17:28, 1508.41s/it]


KeyboardInterrupt: 

In [ ]:
torch.save(en_maddpg, './parameters/weights/en_maddpgs/en_maddpg_{curtime}.pth')